This program will be an example to showcase how to use the edc522, keithley193a (both non SCPI instruments) as well as the arduino model and showcase how to make a rudimentary program

In [1]:
from piec.drivers.utilities import PiecManager
from piec.drivers.stepper_motor.arduino_stepper import Geos_Stepper
from piec.drivers.dc_callibrator.edc522 import EDC522
from piec.drivers.dmm.keithley193a import Keithley193a
pm = PiecManager()


In [2]:
pm.list_resources()

('ASRL3::INSTR', 'GPIB0::9::INSTR', 'GPIB0::10::INSTR')

In [3]:
Stepper = Geos_Stepper("ASRL3::INSTR")
Stepper.idn()

c:\Users\geofr\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyvisa\ctwrapper\highlevel.py:226: VisaIOWarning: VI_WARN_NSUP_ATTR_STATE (1073676420): Although the specified state of the attribute is valid, it is not supported by this implementation.
  return self.handle_return_value(session, ret_value)  # type: ignore


'Custom Arduino_Stepper Object at ASRL3::INSTR'

In [4]:
voltage_program = EDC522("GPIB0::9::INSTR")
voltage_program.idn()

'ID?\r\n000'

In [7]:
voltage_program.set_output(0)

'+0000000'

In [16]:
voltage_program.instrument.write("00000000")

10

In [6]:
voltage_program.set_output(1)

'+1000001'

In [3]:
voltage_out = Keithley193a("GPIB0::10::INSTR")
voltage_out.idn()   # This will print the IDN of the instrument

'NDCV-0000.003E+0\r\n'

In [7]:
voltage_out.read_voltage()

'-0000.005E+0'

In [9]:
Stepper.step(20,1)

VisaIOError: VI_ERROR_SYSTEM_ERROR (-1073807360): Unknown system error (miscellaneous error).